# K-Means Algorithm

The k-means algorithm captures the insight that each point in a cluster should be near to the center of that cluster. It works like this: first we choose k, the number of clusters we want to find in the data. Then, the centers of those k clusters, called centroids.

2 steps :
1. Reassign points step
2. Update centroids step

## 1. Initialization

### Methods for centroid initilization

1. Random selection - randomly pick k data points from the dataset to serve as initial centroids.
2. K means++ - smarter initiliazation that spreads out the centroids and typically leads to better convergence by reducing the likelihood of poor clustering results.


Why initialization matters :
- poor initialization can lead to slow convergence.
- it may trap the algorithm in a local optimum ( we want global optimal clustering solution)

### Random initialization 


In [1]:
import numpy as np

def initialize_centroids(data, k , seed=None):
  """
  Randomly select k unique data points as initial centroids.
  """

  if seed is not None:
    np.random.seed(seed)

  indices = np.random.choice(len(data), size = k, replace = False)
  centroids = data[indices]
  return centroids

# example for low-dimension dataset :

data_low = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])
k = 2
initial_centroids = initialize_centroids(data_low, k, seed=42)
print("Initial Centroids:\n", initial_centroids)

Initial Centroids:
 [[1.  2. ]
 [1.5 1.8]]


### Learning Point:
> The random initialization is simple but may yield different results on each run unless a seed is set. This method may sometimes lead to poor cluster formation. More sophisticated methods like k-means++ can help alleviate this issue.

### Distance Calculation

1. Low-dimensional data ( less than 3 dim ) - eculidean and manhattan distances are often preferred.
2. High-dimensional data ( more than 3 dim) - cosine similarity can be more appropriate, as it takes into account the orientation rather than the magnitude of vectors.

In [2]:
def euclidean_distance(point1, point2):
  return np.sqrt(np.sum((point1 - point2) ** 2))

In [3]:
def manhattan_distance(point1 , point2):
  return np.sum(np.abs(point1 - point2))

In [4]:
point_a = np.array([1, 2])
point_b = np.array([4, 6])
distance = euclidean_distance(point_a, point_b)
print(f"The Euclidean distance between {point_a} and {point_b} is: {distance:.2f}")

The Euclidean distance between [1 2] and [4 6] is: 5.00


In [5]:
point_c = np.array([1, 0, 3])
point_d = np.array([-2, 4, 1])
distance = euclidean_distance(point_c, point_d)
print(f"The Euclidean distance between {point_c} and {point_d} is: {distance:.2f}")

The Euclidean distance between [1 0 3] and [-2  4  1] is: 5.39


### Cosine similarity between vectors x and y is given by :

> sim(x,y) = x.y / || x || || y ||

>  cosine distance = d.cosine(x,y) = 1 - sim(x,y)

In [6]:
def cosine_distance(point1, point2):
  dot_product = np.dot(point1, point2)

  # np.linalg.norm(point) calculates the length of the vector represented by point from the origin of the coordinate system.
  norm1 = np.linalg.norm(point1)
  norm2 = np.linalg.norm(point2)

  # point = np.array([3, 4])
  # underroot( 3^2 + 4^2) = underroot(25) = 5

  if norm1 == 0 or norm2 == 0 :
    return 1.0

  cosine_similarity = dot_product / (norm1 * norm2)
  return 1 - cosine_similarity


## Conditional selection of distance metric


In [7]:
def choose_distance_metric(dim) :
  if dim <= 3 :
    return euclidean_distance, manhattan_distance
  else :
    return cosine_distance, None

In [8]:
# Example for a low-dimensional data point:
point = np.array([1, 2])
print("Euclidean distance from [1, 2] to first centroid:",
      euclidean_distance(point, initial_centroids[0]))
print("Manhattan distance from [1, 2] to first centroid:",
      manhattan_distance(point, initial_centroids[0]))

# Example for a high-dimensional dataset
data_high = np.array([
    [0.1, 0.2, 0.8, 0.9],
    [0.15, 0.22, 0.85, 0.92],
    [0.7, 0.6, 0.3, 0.2]
])
# Using cosine distance as the metric
point_hd = data_high[0]
print("Cosine distance from first high-dim point to second high-dim point:",
      cosine_distance(point_hd, data_high[1]))

Euclidean distance from [1, 2] to first centroid: 0.0
Manhattan distance from [1, 2] to first centroid: 0.0
Cosine distance from first high-dim point to second high-dim point: 0.0008563653798602244


### Learning Point:
1. Euclidean Distance: Sensitive to scale and best for compact, roughly spherical clusters.
2. Manhattan Distance: More robust to outliers in some cases and can capture city-block or grid-like path distances.
3. Cosine Distance: Useful in high-dimensional spaces (e.g., text data) where orientation matters more than magnitude.

## 3. Assignment Step

in this step, every datapoint is assigned to the cluster of the nearest centroid. This is done by calculating the distance from each point to each centroid and choosing choosing the centroid with the minimum distance.

In [14]:
def assign_clusters(data, centroids, distance_func):

  assignments = []
  for point in data:
    # compute distance from point to each centroid.
    distances = [ distance_func(point, centroid) for centroid in centroids ]
    # find the index cluster with the minimum distance
    cluster_index = np.argmin(distances)
    assignments.append(cluster_index)

  return np.array(assignments)

In [ ]:
assignments_low = assign_clusters(data_low, initial_centroids, euclidean_distance)
print("Cluster assignments (Euclidean):", assignments_low)

Cluster assignments (Euclidean): [0 1 1 1 1 1]


In [16]:
assignments_low = assign_clusters(data_low, initial_centroids, manhattan_distance)
print("Cluster assignments (manhattan_distance):", assignments_low)

Cluster assignments (manhattan_distance): [0 1 1 1 0 1]


### Learning point :
Cluster assignment ensures that each data point “belongs” to the nearest centroid, forming the basis for updating centroids in the next iteration.

## Update step:

Once the data points are assigned to clusters, update the centroids by computing the mean of all points belonging to that cluster

In [24]:
def update_centroids(data, assignments, k):
  n_features = data.shape[1]
  new_centroids = np.zeros((k, n_features))
  for idx in range(k):

    if np.any(assignments == idx) :
      new_centroids[idx] = np.mean(data[assignments == idx], axis=0)
    else :
      new_centroids[idx] = data[np.random.choice(range(len(data)))]
  return new_centroids

In [25]:
new_centroids_low = update_centroids(data_low, assignments_low, k)
print("Updated Centroids:\n", new_centroids_low)

Updated Centroids:
 [[1.    1.3  ]
 [5.875 7.2  ]]


### Learning point:

updating centroids to the mean of assigned points is central to the k-means algorithm. It slowly shifts centroid towards the center of their clusters and is a key driver of convergence.

## 5. Iteration and Convergence

K-means clustering is an iterative algorithm. The two primary steps - assignment and update are repeated until :

1. the cluster assigments no longer change, or 
2. a maximum number of iteration is reached

In [26]:
def k_means(data, k , max_iters = 100, tol=1e-4, seed = None):
    """
    Complete k-means algorithm.

    Parameters:
        data (np.array): The dataset.
        k (int): Number of clusters.
        max_iters (int): Maximum number of iterations.
        tol (float): Tolerance for convergence based on centroid movement.
        seed (int): Random seed for reproducibility.

    Returns:
        centroids (np.array): Final centroids.
        assignments (np.array): Final cluster assignments for each data point.
    """
    # initial_centroids
    centroids = initialize_centroids(data, k , seed = seed)

    # choose distance metric based on dimensionality
    dim = data.shape[1]

    if dim <=3 :
        distance_func = euclidean_distance
    else :
        distance_func = cosine_distance

    for iteration in range(max_iters):
        # Assignment step
        assignments = assign_clusters(data, centroids, distance_func)
        # store current centroud for convergence checking
        old_centroids = centroids.copy()
        # update step
        centroids = update_centroids(data, assignments, k)
        # check for convergence (if centroid change less than tol)
        centroid_shifts = np.linalg.norm(centroids - old_centroids, axis = 1)
        if np.all(centroids < tol) :
            print(f"Converged in {iteration} iterations.")
            break

    return centroids, assignments



In [27]:
# Running the algorithm on the low-dimensional dataset:
final_centroids_low, final_assignments_low = k_means(data_low, k, seed=42)
print("Final Centroids (Low-dimensional):\n", final_centroids_low)
print("Final Cluster Assignments (Low-dimensional):", final_assignments_low)

Final Centroids (Low-dimensional):
 [[1.16666667 1.46666667]
 [7.33333333 9.        ]]
Final Cluster Assignments (Low-dimensional): [0 0 1 1 0 1]


In [28]:
# Running the algorithm on the low-dimensional dataset:
final_centroids_low, final_assignments_low = k_means(data_high, k, seed=42)
print("Final Centroids (Low-dimensional):\n", final_centroids_low)
print("Final Cluster Assignments (Low-dimensional):", final_assignments_low)

Final Centroids (Low-dimensional):
 [[0.125 0.21  0.825 0.91 ]
 [0.7   0.6   0.3   0.2  ]]
Final Cluster Assignments (Low-dimensional): [0 0 1]


### Learning Point:
The algorithm’s iterative structure is essential. Convergence is determined when centroids no longer move significantly or after a set number of iterations. However, note that k-means can converge to a local optimum, so sometimes multiple initializations are run to improve results.